# Group 49 : Financial Statment Question Answer Model 

| Name | Student iD | 
|-----------------|--------------|
| GAURAV VIVEK         |         | 
| NISHIT SRIVASTAVA            |           |
|JITENDRA KUMAR ||
| GAURAVA KUMAR KATIYAR            |           |
| VINOD KUMAR SINGH           |           |


We have consider 2024-25 Consolidated and Standalone statemnet of TCS .Link give below 

https://www.tcs.com/content/dam/tcs/investor-relations/financial-statements/2024-25/q4/IND%20AS/Consolidated%20and%20Standalone.pdf

This has financial statment of of 2024-25(Year End March 31 2025 ) and 2023-24 (Year End March 31 2024 )
We  are considering Consolidated statement for our modeling

In [1]:
from pypdf import PdfReader
import pandas as pd
import requests
from io import BytesIO
import re
import uuid
from typing import List, Dict
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
from sentence_transformers import SentenceTransformer
import faiss
from rank_bm25 import BM25Okapi
import numpy as np
from sklearn.preprocessing import minmax_scale
from transformers import (
    pipeline,
    T5Tokenizer, T5ForConditionalGeneration,
    AutoTokenizer, AutoModelForSeq2SeqLM
)
import pickle
from rapidfuzz import fuzz

from nltk.corpus import stopwords
import json
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

# Download stopwords if not already
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /home/gaurav/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/gaurav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/gaurav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1. Data Collection & Preprocessing

### 1.1 Reading Pdf

In [2]:

pdf_url = "https://www.tcs.com/content/dam/tcs/investor-relations/financial-statements/2024-25/q4/IND%20AS/Consolidated%20and%20Standalone.pdf"
response = requests.get(pdf_url)
response.raise_for_status() 
pdf_file = BytesIO(response.content)
reader = PdfReader(pdf_file)

### 1.2 Segment Report

In [3]:
# Initialize dictionary to collect text
annual_result = {
    "financial_results": "",
    "segment_information": "",
    "balance_sheet": "",
    "cash_flows": ""
}

# Extract and classify text
for page in reader.pages:
    text = page.extract_text()
    if text:
        text_lower = text.lower()
        
        if "consolidated" in text_lower and "financial results" in text_lower:
            annual_result["financial_results"] += text + "\n"
        
        if "consolidated" in text_lower and "segment information" in text_lower:
            annual_result["segment_information"] += text + "\n"
        
        if "consolidated" in text_lower and "balance sheet" in text_lower:
            annual_result["balance_sheet"] += text + "\n"
        
        if "consolidated" in text_lower and "cash flows" in text_lower:
            annual_result["cash_flows"] += text + "\n"

# Convert dictionary to DataFrame
df_annual_result = pd.DataFrame({
    "Segment": list(annual_result.keys()),
    "Data": list(annual_result.values())
})
df_annual_result

,Segment,Data
0,financial_results,"\nMarch 31, December 31, March 31, March 31,..."
1,segment_information,"March 31, December 31, March 31, March 31, Mar..."
2,balance_sheet,"As at As at\nMarch 31, 2025 March 31, 2024\nAS..."
3,cash_flows,\nSelect explanatory notes to the Statement o...


### 1.3 DataCleanup

#### 1.3.1 Split the Data column 

In [4]:
df_annual_result = df_annual_result.assign(
    Data=df_annual_result["Data"].str.split("\n")
).explode("Data").reset_index(drop=True)
df_annual_result

,Segment,Data
0,financial_results,
1,financial_results,"March 31, December 31, March 31, March 31, Mar..."
2,financial_results,2025 2024 2024 2025 2024
3,financial_results,"Revenue from operations 64,4..."
4,financial_results,"Other income 1,028 1,243 ..."
...,...,...
318,cash_flows,Audited Consolidated Statement of Cash Flows
319,cash_flows,(` crore)
320,cash_flows,
321,cash_flows,*Represents value less than `0.50 crore.


#### 1.3.2 Remove Extrat space dot and bracket

In [5]:
df_annual_result["Data"] = (
    df_annual_result["Data"]
    .str.replace(r"[.,()]", "", regex=True)  # remove . , ( )
    .str.split()                             # split on any whitespace
    .str.join(" ")                           # rejoin with single spaces
)


#### 1.3.3 Split string into table format

In [6]:
def split_data(row):
    parts = row["Data"].split()
    n_extra = 5  

    if row["Segment"] in ["financial_results", "segment_information"]:
        # Last 5 parts go into col1–col5
        last_parts = parts[-5:]
        first_part = " ".join(parts[:-5])
        return pd.Series([first_part] + last_parts)

    elif row["Segment"] in ["balance_sheet", "cash_flows"]:
        # Last 2 parts go into col4 and col5
        last_parts = parts[-2:]
        first_part = " ".join(parts[:-2])
        return pd.Series([first_part] + [None, None, None] + last_parts)

    else:
        return pd.Series([row["Data"]] + [None] * n_extra)



# Apply with different column counts
df_split = df_annual_result.apply(split_data, axis=1)

# Rename columns dynamically for clarity
df_split.columns = ["attribute", "col1", "col2", "col3", "2025", "2024"]

# Combine with original DataFrame if needed
df_result = pd.concat([df_annual_result, df_split], axis=1)

df_result

,Segment,Data,attribute,col1,col2,col3,2025,2024
0,financial_results,,,NaN,NaN,NaN,NaN,NaN
1,financial_results,March 31 December 31 March 31 March 31 March 31,March 31 December 31 March,31,March,31,March,31
2,financial_results,2025 2024 2024 2025 2024,,2025,2024,2024,2025,2024
3,financial_results,Revenue from operations 64479 63973 61237 2553...,Revenue from operations,64479,63973,61237,255324,240893
4,financial_results,Other income 1028 1243 1157 3962 4422,Other income,1028,1243,1157,3962,4422
...,...,...,...,...,...,...,...,...
318,cash_flows,Audited Consolidated Statement of Cash Flows,Audited Consolidated Statement of,None,None,None,Cash,Flows
319,cash_flows,` crore,,None,None,None,`,crore
320,cash_flows,,,None,None,None,NaN,NaN
321,cash_flows,*Represents value less than `050 crore,*Represents value less than,None,None,None,`050,crore


#### 1.3.4 Keeping row having interger in 2025 and 2024 column

In [7]:
def is_int(val):
    try:
        return float(val).is_integer()
    except (ValueError, TypeError):
        return False

# Keep only rows where both are integers
df_result= df_result[df_result["2025"].apply(is_int)]
df_result= df_result[df_result["2024"].apply(is_int)]
df_result.drop(columns=["col1","col2","col3","Data"], inplace=True)

### 2. Retrieval-Augmented Generation (RAG) System Implementation

#### 2.1 Data Processing

#### 2.1.1 Combining all data for chunk creation 

In [8]:
text_rows = []
for _, row in df_result.iterrows():
    line_item = row["attribute"]
    for year in ["2024"]:
        value = row[year]
        if pd.notna(value):
            text_rows.append(f"{line_item} in {year} was {value}")
    for year in ["2025"]:
        value = row[year]
        if pd.notna(value):
            text_rows.append(f"{line_item} in {year} was {value}")


#### 2.1.2 Creating chunk with token size as 100 and 400 

In [9]:


def create_chunks(text_rows, chunk_size=100):
    chunks = []
    current_chunk = ""

    for row in text_rows:
        if len(current_chunk) + len(row) + 1 <= chunk_size:
            current_chunk += row + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = row + " "

    if current_chunk:  # Add last chunk
        chunks.append(current_chunk.strip())

    return chunks

# Example
chunks_100 = create_chunks(text_rows, chunk_size=100)
chunks_400 = create_chunks(text_rows, chunk_size=400)

all_chunks = chunks_100 + chunks_400
print(f"Created {len(all_chunks)} chunks")

Created 219 chunks


### 2.2 Embedding & Indexing

For **Embedding** uisng **all-MiniLM-L6-v2**

For **Dence Vector store** using **Faiss**

For **Spare Index** using **BM25**

In [10]:

embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

dense_embeddings = embedding_model.encode([chunk for chunk in all_chunks], convert_to_numpy=True)
dimension = dense_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(dense_embeddings)

tokenized_corpus = [word_tokenize(chunk.lower()) for chunk in all_chunks]
bm25 = BM25Okapi(tokenized_corpus)


Storing Faiss, bm25 and document chunkd for future use 

In [11]:
faiss.write_index(faiss_index, "faiss_index.idx")


with open("bm25.pkl", "wb") as f:
    pickle.dump(bm25, f)

with open("all_chunks.pkl", "wb") as f:
    pickle.dump(all_chunks, f)

### 2.3 Hybrid Retrieval Pipelin with Hybrid Search (Sparse + Dense Retrieval)

**1.Preprocess question** - Converts query to lowercase. Removes special characters.

**2.Compute dense vector similarity using FAISS.** - Get the Question embedding,search top 5 chunk and get id and score for chunk
    
**3.Compute BM25 sparse keyword relevance.** -Get word tokennizer,search top 5 chunk and get id and score for chunk 
    
**4.Normalize scores for both methods.**
    
**5.Fuse scores with weighted sum (alpha)**
    
**6.Rank by hybrid score and return top-N results.**

In [12]:

def hybrid_search(query: str, top_n: int = 5, alpha: float = 0.5):
    # Preprocess query
    query_clean = re.sub(r"[^a-zA-Z0-9\s]", "", query.lower())
    

    query_embedding = embedding_model.encode([query_clean], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)
    dense_scores, dense_indices = faiss_index.search(query_embedding, top_n)
    dense_results = [(int(idx), float(score)) for idx, score in zip(dense_indices[0], dense_scores[0])]
    
    # Sparse retrieval
    sparse_scores = bm25.get_scores(word_tokenize(query_clean))
    sparse_indices = np.argsort(sparse_scores)[::-1][:top_n]
    sparse_results = [(int(idx), float(sparse_scores[idx])) for idx in sparse_indices]
    
   
    dense_dict = {idx: s for idx, s in dense_results}
    sparse_dict = {idx: s for idx, s in sparse_results}
    dense_norm = minmax_scale(list(dense_dict.values())) if dense_dict else []
    sparse_norm = minmax_scale(list(sparse_dict.values())) if sparse_dict else []
    for i, (idx, _) in enumerate(dense_results):
        dense_dict[idx] = dense_norm[i]
    for i, (idx, _) in enumerate(sparse_results):
        sparse_dict[idx] = sparse_norm[i]
        
        # Weighted fusion
    scores_dict = {}
    for idx, score in dense_dict.items():
        scores_dict[idx] = scores_dict.get(idx, 0) + alpha * score
    for idx, score in sparse_dict.items():
        scores_dict[idx] = scores_dict.get(idx, 0) + (1 - alpha) * score
        
    fused_scores = sorted(scores_dict.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    return [(all_chunks[idx], score) for idx, score in fused_scores]




### 2.5 Guardrail Implementation

#### 2.5.1 Get Finance realted word for relavent question

In [13]:

stop_words = set(stopwords.words("english"))

def clean_chunks(chunks):
    unique_words = set()
    
    for chunk in chunks:
        tokens = word_tokenize(chunk.lower())
        for token in tokens:
            # Remove stopwords
            if token in stop_words:
                continue
            # Remove digits and words containing digits
            if re.search(r'\d', token):
                continue
            # Keep only alphabetic tokens
            if token.isalpha():
                unique_words.add(token)
    
    return list(unique_words)


# Example chunks
with open("all_chunks.pkl", "rb") as f:
    chunked_docs = pickle.load(f)


FINANCE_KEYWORDS = clean_chunks(chunked_docs)


#### 2.5.2 Validation Input Question 

In [14]:

HARMFUL_KEYWORDS = [
    "kill", "hate", "bomb", "hack", "terror", 'password', 'credit card',
    'social security', 'ssn', 'account number', 'routing number', 'login',
    'credentials', 'hack', 'exploit', 'bypass', 'unauthorized', 'illegal',
    'scam', 'phishing', 'malware', 'virus'
]



def is_fuzzy_match(word, keywords, threshold=80):
    """Return True if word is close enough to any keyword."""
    for keyword in keywords:
        if fuzz.ratio(word.lower(), keyword.lower()) >= threshold:
            return True
    return False

def validate_query(query):
    query = query.strip()
    if not query:
        return False, {"reason": "Query is Empty "}

    tokens = re.findall(r"\w+", query.lower())  # split into words

    # Length filter (less than 4 words)
    if len(tokens) < 4:
        return False, {"reason": "Query is very short"}

    # Harmful filter
    if any(is_fuzzy_match(token, HARMFUL_KEYWORDS, 95) for token in tokens):
        return False, {"reason": "Query contain harmful word "}

    # Finance filter
    if not any(is_fuzzy_match(token, FINANCE_KEYWORDS) for token in tokens):
        return False, {"reason": "Query is not relevent "}

    return True, {"reason": "valid"}


In [15]:
def validate_ans(context,answer,threshold=50):
    answer = answer.strip()
    if not answer:
        return False, {"reason": "Query is Empty "}

    tokens = re.findall(r"\w+", answer.lower())  # split into words


    # Harmful filter
    if any(is_fuzzy_match(answer, HARMFUL_KEYWORDS, 95) for token in tokens):
        return False, {"reason": "Query contain harmful word "}

    # Finance filter
    score = fuzz.partial_ratio(answer.lower(), context.lower())
    if score < threshold:
        answer+= " ,Answer is not present in context "
        return True, {"answer":  answer  }

    return True, {"answer":  answer  }



### 2.5 Response Generation

We are using T5-small model for response generation

In [16]:
pretrain_tokenizer = T5Tokenizer.from_pretrained("t5-small")
pretrain_model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Limit total input tokens to the model context window.

In [17]:
def count_tokens(text, tokenizer):
    """Return number of tokens for a given text."""
    return len(tokenizer.encode(text))


def build_messages(context, question, tokenizer, max_ctx=4096, reserve=512):
    """
    Build input prompt while keeping total tokens <= max_ctx.
    Reserve some space for the model output.
    """
    # Token counts
    question_tokens = tokenizer.encode(question)
    context_tokens = tokenizer.encode(context)

    # budget for context
    budget = max_ctx - (len(question_tokens) + reserve)

    # truncate context if needed
    if len(context_tokens) > budget:
        context_tokens = context_tokens[:budget]
        context = tokenizer.decode(context_tokens)

    input_text = f"question: {question} \ncontext: {context} "
    return input_text


In [18]:




def generate_ans(question, tokenizer, model, max_ctx=4096, reserve=512):
    """
    Retrieve context, build prompt, and generate within token limits.
    """
    is_valid, result = validate_query(question)
    if not is_valid:
        return(result["reason"])
    results = hybrid_search(question, top_n=5, alpha=0.6)

    # Concatenate retrieved documents
    context = ""
    for text, score in results:
        context += "\n " + text.lower()

    # Build input within token budget
    input_text = build_messages(context, question, tokenizer, max_ctx=max_ctx, reserve=reserve)

    # Final encoding + generation
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_ctx-reserve)
    outputs = model.generate(**inputs, max_length=reserve)  # reserve tokens for answer
    is_valid, ans=validate_ans(context,tokenizer.decode(outputs[0], skip_special_tokens=True))
    
    
        
    return ans['answer']


In [19]:
import time
test_questions = [
    "What was TCS’s consolidated net profit in FY 2024?",
"What was the total comprehensive income for FY 2024?",
"Which segment had the highest revenue in FY 2024?",
"What was TCS’s total assets as on March 31 2024?",
"How much cash and cash equivalents did TCS hold at year-end 2024?",
"What were employee benefit expenses in FY 2025?",
"How much did TCS spend on equipment and software licences in FY 2025?",
"What were the total expenses for FY 2025?",
"What was profit before tax (PBT) in FY 2025?"
]

expected_answers = [
  "46099",
"46336",
"Banking Financial Services & Insurance (90928 )",
"146449",
"9016",
"145788",
"11648",
"193955",
"65331"
]
results = []

for q, expected in zip(test_questions, expected_answers):
    start_time = time.time()
    output_text=generate_ans(q,pretrain_tokenizer,pretrain_model)
    end_time = time.time()
    inference_time = round((end_time - start_time) , 2)
    accuracy = "Yes" if expected.lower() in output_text.lower() else "No"  

    
    results.append({
        "Question": q,
        "Expected Ans": expected,
        "Model Output": output_text,
        "Correct": accuracy,
        "Inference Time (ms)": inference_time
    })
    #print(results)

df = pd.DataFrame(results)


In [20]:
df

,Question,Expected Ans,Model Output,Correct,Inference Time (ms)
0,What was TCS’s consolidated net profit in FY 2...,46099,46099 profit,Yes,0.17
1,What was the total comprehensive income for FY...,46336,46336,Yes,0.15
2,Which segment had the highest revenue in FY 2024?,Banking Financial Services & Insurance (90928 ),2025,No,0.12
3,What was TCS’s total assets as on March 31 2024?,146449,31 total equity and liabilities in 2025 was 15...,No,0.70
4,How much cash and cash equivalents did TCS hol...,9016,9016,Yes,0.10
5,What were employee benefit expenses in FY 2025?,145788,140131,No,0.11
6,How much did TCS spend on equipment and softwa...,11648,3702,No,0.15
7,What were the total expenses for FY 2025?,193955,182360 total comprehensive income for the year...,No,0.88
8,What was profit before tax (PBT) in FY 2025?,65331,61997 profit before tax in 2024 was 65331 prof...,Yes,1.52


In [21]:
#! pip install rapidfuzz

## 3 Fine-Tuned Model System Implementation

We have selected Retrieval-Augmented Fine-Tuning , and thus we will pass Question, context and expected answer to the model to fine tune it . We have craeted jsonl file with these deatils and being used here 

### 3.1 dataset Preperation 

In [22]:
dataset = load_dataset("json", data_files="qa_with_context.jsonl")
# convert the data set to t5 formart 
def format_for_t5(example):
    return {
        "input_text": f"question: {example['question']} context: {example['context']}",
        "target_text": example["answer"]
    }

formatted_dataset = dataset["train"].map(format_for_t5)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

Tokennized the data

In [23]:
def preprocess(example):
    model_inputs = tokenizer(
        example["input_text"], 
        max_length=512, 
        truncation=True
    )
    labels = tokenizer(
        example["target_text"], 
        max_length=128, 
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to formatted dataset
tokenized = formatted_dataset.map(preprocess, batched=True)

In [24]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Set dataset format for PyTorch
tokenized.set_format(
    type="torch", 
    columns=["input_ids", "attention_mask", "labels"]
)
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
# Training arguments
training_args = TrainingArguments(
    output_dir="./t5_rag_finetuned",
    eval_strategy="epoch",            
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    save_strategy="epoch",             
    logging_steps=5
)

/media/gaurav/ubuntudata/virualenv/CAI_assigmnet/lib/python3.12/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [25]:
import torch

print("===== Training Hyperparameters =====")
print(f"Learning rate: {training_args.learning_rate}")
print(f"Train batch size: {training_args.per_device_train_batch_size}")
print(f"Eval batch size: {training_args.per_device_eval_batch_size}")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Weight decay: {training_args.weight_decay}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("====================================\n")


===== Training Hyperparameters =====
Learning rate: 0.0003
Train batch size: 8
Eval batch size: 8
Epochs: 5
Weight decay: 0.01
Device: CPU



In [26]:

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    eval_dataset=tokenized,  
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train
trainer.train()


/tmp/ipykernel_67567/941882172.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/media/gaurav/ubuntudata/virualenv/CAI_assigmnet/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/media/gaurav/ubuntudata/virualenv/CAI_assigmnet/lib/python3.12/site-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,2.160600,1.159647
2,2.093300,0.809321
3,0.831400,0.670757
4,0.696700,0.600975
5,0.542900,0.570303


/media/gaurav/ubuntudata/virualenv/CAI_assigmnet/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/media/gaurav/ubuntudata/virualenv/CAI_assigmnet/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/media/gaurav/ubuntudata/virualenv/CAI_assigmnet/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/media/gaurav/ubuntudata/virualenv/CAI_assigmnet/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory w

TrainOutput(global_step=35, training_loss=1.216676003592355, metrics={'train_runtime': 135.3329, 'train_samples_per_second': 1.884, 'train_steps_per_second': 0.259, 'total_flos': 17965831127040.0, 'train_loss': 1.216676003592355, 'epoch': 5.0})

In [29]:
finetune_model=model
finetune_tokenizer=tokenizer
model.save_pretrained("./t5-small-qa-model")
tokenizer.save_pretrained("./t5-small-qa-model")


('./t5-small-qa-model/tokenizer_config.json',
 './t5-small-qa-model/special_tokens_map.json',
 './t5-small-qa-model/spiece.model',
 './t5-small-qa-model/added_tokens.json')

In [47]:
import time
test_questions = [
    "What was TCS’s consolidated net profit in FY 2024?",
"What was the total comprehensive income for FY 2024?",
"Which segment had the highest revenue in FY 2024?",
"What was TCS’s total assets as on March 31 2024?",
"How much cash and cash equivalents did TCS hold at year-end 2024?",
"What were employee benefit expenses in FY 2025?",
"How much did TCS spend on equipment and software licences in FY 2025?",
"What were the total expenses for FY 2025?",
"What was profit before tax (PBT) in FY 2025?",
"who alll play in IPL ?",
"How to hack the system "
]

expected_answers = [
  "46099",
"46336",
"Banking Financial Services & Insurance",
"146449",
"9016",
"145788",
"11648",
"193955",
"65331",
"Query is not relevnet",
"Query contain harmful word"
]
results = []

for q, expected in zip(test_questions, expected_answers):
    ## pretrain model 
    start_time = time.time()
    output_text=generate_ans(q,pretrain_tokenizer,pretrain_model)
    end_time = time.time()
    inference_time = round((end_time - start_time) , 2)
    accuracy = "Yes" if expected.lower() in output_text.lower() else "No"  

    
    results.append({
        "Question": q,
        "Method": "RAG",
        "Model Output": output_text,
        "Correct": accuracy,
        "Inference Time (ms)": inference_time
    })

    ## fine rune model 
    start_time = time.time()
    output_text=generate_ans(q,finetune_tokenizer,finetune_model)
    end_time = time.time()
    inference_time = round((end_time - start_time) , 2)
    
    accuracy = "Yes" if (expected.lower() in output_text.lower() or expected.lower() == output_text.lower()) else "No"
 

    
    results.append({
        "Question": q,
        "Method": "Fine-Tune",
        "Model Output": output_text,
        "Correct": accuracy,
        "Inference Time (ms)": inference_time
    })
    #print(results)

df = pd.DataFrame(results)


In [48]:
df

,Question,Method,Model Output,Correct,Inference Time (ms)
0,What was TCS’s consolidated net profit in FY 2...,RAG,46099 profit,Yes,0.27
1,What was TCS’s consolidated net profit in FY 2...,Fine-Tune,46099,Yes,0.19
2,What was the total comprehensive income for FY...,RAG,46336,Yes,0.14
3,What was the total comprehensive income for FY...,Fine-Tune,46336,Yes,0.17
4,Which segment had the highest revenue in FY 2024?,RAG,2025,No,0.13
5,Which segment had the highest revenue in FY 2024?,Fine-Tune,Financial Services Services (Security and Insu...,No,0.26
6,What was TCS’s total assets as on March 31 2024?,RAG,31 total equity and liabilities in 2025 was 15...,No,0.71
7,What was TCS’s total assets as on March 31 2024?,Fine-Tune,12267,No,0.12
8,How much cash and cash equivalents did TCS hol...,RAG,9016,Yes,0.11
9,How much cash and cash equivalents did TCS hol...,Fine-Tune,9016,Yes,0.11
